In [1]:
import pandas
import json
import nltk
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api

In [2]:
with open('data.json') as f:
    meta_data = json.load(f)
print(len(meta_data))

42


In [4]:
type(meta_data)

list

In [9]:
meta_data[40].items()

dict_items([('grow_cycle_id', 931), ('family_name', 'Lettuce'), ('category', 'Romaine'), ('date', ['2023-10-24']), ('s3_path', ''), ('annotation_type', []), ('status', 'download and image selection are completed, Yet to verified the QC.')])

In [12]:
grow_cycle_id=[]
family_name=[]
category=[]
annotation_type=[]
for i in meta_data:
    grow_cycle_id.append(i['grow_cycle_id'])
    family_name.append(i['family_name'])
    category.append(i['category'])
    annotation_type.append(i['annotation_type'])
print("grow_cycle_id:", grow_cycle_id)
print("family_name:", family_name)
print("category:", category)
print("annotation_type:", annotation_type)

grow_cycle_id: [51, 52, 53, 54, 55, 57, 61, 93, 95, 96, 115, 116, 119, 121, 144, 146, 148, 149, 151, 216, 218, 219, 220, 224, 249, 250, 251, 252, 253, 255, 256, 875, 876, 882, 883, 899, 900, 911, 912, 929, 931, '']
family_name: ['Basil', 'Basil', 'Lettuce', 'Basil', 'Lettuce', 'Lettuce', 'Lettuce', 'basil', 'basil', 'Lettuce', 'Basil', 'Lettuce', 'basil', 'basil', 'Cucumber', 'Cucumber', 'Cucumber', 'Basil', 'Basil', 'strawberry', 'Lettuce', 'Arugula', 'Arugula', 'Lettuce', 'Basil', 'Arugula', 'Pac Chai', 'kale', 'Basil', 'Pac choi', 'kale', 'Lettuce', 'Lettuce', 'Lettuce', 'Basil', 'Arugula', 'Basil', 'Lettuce', 'Lettuce', 'Basil', 'Lettuce', '']
category: ['zinc deficient-basil', 'full nutrient-basil (mrs-burn-lemon)', 'full nutrient-green butter', 'zinc deficient + treatment basil', 'Zinc deficient + treatment-green butter', 'iron deficient-green butter', 'Demo Scan', 'iron deficient basil', 'iron deficient + treatment basil', 'iron deficient + treatment green butter', 'Nufar Full N

In [14]:
print(meta_data[0]['grow_cycle_id'])
print(meta_data[0]['family_name'])
print(meta_data[0]['category'])
print(meta_data[0]['annotation_type'])

51
Basil
zinc deficient-basil
['annotated-data/moonshot/rgb/70/box/basil/mrs-burns-lemon/disease/zinc/deficiency/', 'annotated-data/moonshot/rgb/70/box/basil/mrs-burns-lemon/disease/zinc/leaf/', 'annotated-data/moonshot/rgb/70/box/basil/mrs-burns-lemon/disease/zinc/plant/']


In [10]:
words=[]
for sent in family_name:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [11]:
words

[['basil'],
 ['basil'],
 ['lettuce'],
 ['basil'],
 ['lettuce'],
 ['lettuce'],
 ['lettuce'],
 ['basil'],
 ['basil'],
 ['lettuce'],
 ['basil'],
 ['lettuce'],
 ['basil'],
 ['basil'],
 ['cucumber'],
 ['cucumber'],
 ['cucumber'],
 ['basil'],
 ['basil'],
 ['strawberry'],
 ['lettuce'],
 ['arugula'],
 ['arugula'],
 ['lettuce'],
 ['basil'],
 ['arugula'],
 ['pac', 'chai'],
 ['kale'],
 ['basil'],
 ['pac', 'choi'],
 ['kale'],
 ['lettuce'],
 ['lettuce'],
 ['lettuce'],
 ['basil'],
 ['arugula'],
 ['basil'],
 ['lettuce'],
 ['lettuce'],
 ['basil'],
 ['lettuce']]